In [224]:
import pandas as pd
from sqlite3 import connect

In [225]:
results = pd.read_csv("data/results.csv", dtype={'basho': 'string', 'day': 'string', 'rikishi1_id': 'int64', 'rikishi1_rank': 'string', 'rikishi1_shikona': 'string', 'rikishi1_result': 'string', 'rikishi1_win': 'int64', 'kimarite': 'string', 'rikishi2_id': 'int64', 'rikishi2_rank': 'string', 'rikishi2_shikona': 'string', 'rikishi2_result': 'string', 'rikishi2_win': 'string'})
results.head()

,basho,day,rikishi1_id,rikishi1_rank,rikishi1_shikona,rikishi1_result,rikishi1_win,kimarite,rikishi2_id,rikishi2_rank,rikishi2_shikona,rikishi2_result,rikishi2_win
0,1983.01,1,4140,J13w,Chikubayama,0-1 (7-8),0,yorikiri,4306,Ms1e,Ofuji,1-0 (6-1),1
1,1983.01,1,4306,Ms1e,Ofuji,1-0 (6-1),1,yorikiri,4140,J13w,Chikubayama,0-1 (7-8),0
2,1983.01,1,1337,J12w,Tochitsukasa,1-0 (9-6),1,oshidashi,4323,J13e,Shiraiwa,0-1 (3-12),0
3,1983.01,1,4323,J13e,Shiraiwa,0-1 (3-12),0,oshidashi,1337,J12w,Tochitsukasa,1-0 (9-6),1
4,1983.01,1,4097,J12e,Tamakiyama,0-1 (8-7),0,yorikiri,4319,J11w,Harunafuji,1-0 (5-10),1


In [226]:
banzuke = pd.read_csv("data/banzuke.csv", dtype={'basho': 'string', 'id': 'int64', 'rank': 'string', 'rikishi': 'string', 'heya': 'string', 'susshin': 'string', 'birth_date': 'string', 'height': 'float64', 'weight': 'float64', 'birth_date': 'string', 'prev': 'string', 'prev_w': 'float64', 'prev_l': 'float64'})
banzuke = banzuke.drop (['basho', 'prev', 'prev_w', 'prev_l'], axis=1) # I don't need these columns for the final analysis.
banzuke = banzuke.drop (['rank', 'rikishi'], axis=1) # These are important columns, but they are shared with results.csv and are being cleaned up with that file. 
banzuke

,id,heya,shusshin,birth_date,height,weight
0,1354,Kokonoe,Hokkaido,1955-06-01,182.0,116.0
1,4080,Mihogaseki,Hokkaido,1953-05-16,179.0,165.0
2,4095,Futagoyama,Aomori,1953-04-03,186.0,133.0
3,4104,Futagoyama,Aomori,1952-09-29,181.0,144.0
4,4112,Sadogatake,Mie,1957-04-26,183.0,163.0
...,...,...,...,...,...,...
177873,12833,Onoe,Okinawa,2000-05-16,180.0,175.0
177874,12734,Shikihide,Ibaraki,2006-08-15,180.0,115.0
177875,12631,Sakaigawa,Chiba,2005-01-05,168.0,128.0
177876,2905,Kise,Kumamoto,1988-01-27,172.0,113.0


In [227]:
# I have already cleaned up the rank and rikishi/shikona (wrestler/ring name) information in the other CSV file, so I will now
# start cleaning up the shusshin (home country/prefecture) column. These steps are similar to the method wherein I tied ID numbers to shikona.

rikishi_shusshin_list = []

for i in banzuke["shusshin"].unique():
    rikishi_shusshin_list.append(i)

# Next to make a dictionary that holds each wrestler's unique ID number as a key, with values being each shusshin associated with that wrestler.
# The vast majority of ID numbers will only feature one shusshin, but this will make sure to include any additional shusshin entries as well.

rikishi_shusshin_dict = {}

# Initializes a key for every number results["id"] but still only adds one shusshin as a value per key, overwriting old shusshin.
# When I was trying to discover all the shikona (ring names) for each wrestler, it was important to preserve all the older names. That is different from what
# I want to accomplish with the shusshin entries. Most wrestler's don't change their shusshin, but a few of them do, ostensibly from moving from one prefecture in
# Japan to another. (Wrestlers from abroad still seem to keep their original countries as shusshin, even when becoming Japanese citizens.)
# Unlike with the shikona, I'm not going to completely fill out this dictionary will all shusshin; just a wrestler's current shusshin will suffice for analysis purposes.
for i in range(len(banzuke)):
    rikishi_shusshin_dict.update({banzuke["id"][i]: [banzuke["shusshin"][i]]})

# Now I turn this into a new dataframe.
shusshin_df = pd.DataFrame.from_dict(
    [([key] + value) for key, value in rikishi_shusshin_dict.items()]
)
shusshin_df.rename(columns={0: "id", 1: "shusshin"}, inplace=True)

# shusshin_df["shusshin"][0] = "wow"
shusshin_df.head()

,id,shusshin
0,1354,Hokkaido
1,4080,Hokkaido
2,4095,Aomori
3,4104,Aomori
4,4112,Mie


In [228]:
# I now do the same thing with heya (sumo stable) as I did with shusshin.

rikishi_heya_list = []

for i in banzuke["heya"].unique():
    rikishi_heya_list.append(i)

# Next to make a dictionary that holds each wrestler's unique ID number as a key, with values being each shusshin associated with that wrestler.
# The vast majority of ID numbers will only feature one shusshin, but this will make sure to include any additional shusshin entries as well.

rikishi_heya_dict = {}

for i in range(len(banzuke)):
    rikishi_heya_dict.update({banzuke["id"][i]: [banzuke["heya"][i]]})

# If I wanted to show all the different heya each wrestler had ever belonged to, I would add the following deactivated code.
# But, as with shusshin,this analysis is only concerned with the most recent heya to which each wrestler has belonged.

# for index, row in banzuke.iterrows():
#     if row["heya"] not in rikishi_heya_dict[row["id"]]:
#         rikishi_heya_dict[row["id"]].append(row["heya"])

heya_df = pd.DataFrame.from_dict(
    [([key] + value) for key, value in rikishi_heya_dict.items()]
)
heya_df.rename(columns={0: "id", 1: "heya"}, inplace=True)
heya_df

,id,heya
0,1354,Kokonoe
1,4080,Mihogaseki
2,4095,Futagoyama
3,4104,Futagoyama
4,4112,Sadogatake
...,...,...
5277,12833,Onoe
5278,12834,Ajigawa
5279,12838,Naruto
5280,12837,Sadogatake


In [229]:
# I now do the same thing with height as I did with shusshin and heya. As with heya, I am only concerned with the most recent entry for height,
# which is why I am not populating the dictionary with all values associated with each wrestler's height.
# I imagine most wrestlers in the top division, having been wrestling for years, are adults with heights that do not fluctuate much.

rikishi_height_list = []

for i in banzuke["height"].unique():
    rikishi_height_list.append(i)

rikishi_height_dict = {}

for i in range(len(banzuke)):
    rikishi_height_dict.update({banzuke["id"][i]: [banzuke["height"][i]]})

height_df = pd.DataFrame.from_dict(
    [([key] + value) for key, value in rikishi_height_dict.items()]
)
height_df.rename(columns={0: "id", 1: "height"}, inplace=True)
height_df

,id,height
0,1354,183.0
1,4080,179.0
2,4095,186.0
3,4104,181.5
4,4112,183.5
...,...,...
5277,12833,180.0
5278,12834,175.0
5279,12838,183.0
5280,12837,178.0


In [230]:
# I now use the same approach to make a dataframe with each wrestler's weight. Unlike with some of the other columns,

rikishi_weight_list = []

for i in banzuke["weight"]: # because I want an average of all weights each wrester has been listed at during these tournaments, I do not want to use .unique() here.
    rikishi_weight_list.append(i)

rikishi_weight_dict = {}

for i in range(len(banzuke)):
    rikishi_weight_dict.update({banzuke["id"][i]: [banzuke["weight"][i]]})

for index, row in banzuke.iterrows():
    if row["weight"] not in rikishi_weight_dict[row["id"]]:
        rikishi_weight_dict[row["id"]].append(row["weight"])

# Now that I have a dictionary listing each wrestler's ID with the differing weights that wrestler had during tournaments, now I want to create a dictionary
# that features an average of all those weights.

def average_weight(weight_list):
    return round(sum(weight_list) / len(weight_list), 1)

average_weight_dict = {}

for key, value in rikishi_weight_dict.items():
    if average_weight(value) > 0: 
        value = [average_weight(value)]
    else:
        value = [0] # There are some wrestlers who don't have an average weight, because they have no entries in the "weight" column. I will mark null entries as "0".
    average_weight_dict[key] = value

average_weight_dict



{1354: [121.3],
 4080: [167.0],
 4095: [133.0],
 4104: [150.6],
 4112: [168.2],
 4127: [122.8],
 1363: [142.8],
 1379: [178.6],
 4116: [123.7],
 4134: [159.8],
 4078: [142.4],
 1374: [137.1],
 4099: [144.4],
 4126: [130.3],
 1369: [149.5],
 1356: [157.4],
 4107: [145.1],
 4102: [149.3],
 1353: [155.3],
 4094: [142.8],
 1375: [148.2],
 4077: [138.7],
 1338: [142.8],
 4133: [131.7],
 1378: [142.7],
 4117: [123.5],
 4050: [200.3],
 1352: [136.9],
 4132: [146.5],
 4100: [139.0],
 1345: [120.7],
 1334: [145.0],
 4118: [173.2],
 4124: [121.5],
 1348: [138.0],
 4131: [113.8],
 4122: [129.0],
 4103: [132.7],
 1342: [135.4],
 4091: [109.7],
 4087: [126.0],
 1350: [181.0],
 1359: [142.4],
 1380: [136.2],
 4128: [120.8],
 1332: [122.6],
 4322: [116.5],
 4311: [127.4],
 1347: [134.4],
 1385: [127.5],
 4125: [134.5],
 4316: [118.4],
 1361: [122.8],
 4318: [143.3],
 1302: [151.1],
 4130: [108.0],
 4111: [134.0],
 4109: [112.0],
 4129: [113.0],
 4319: [158.5],
 4097: [134.0],
 1337: [146.1],
 4323: [

In [231]:
# Now I turn this avergae weight into a new dataframe.
weight_df = pd.DataFrame.from_dict([([key] + value) for key, value in average_weight_dict.items()])
weight_df.rename(columns={0: "id", 1: "avg_weight"}, inplace=True)
weight_df.head()

,id,avg_weight
0,1354,121.3
1,4080,167.0
2,4095,133.0
3,4104,150.6
4,4112,168.2


In [232]:
# For the sake of completeness, I am going to clean up the birth_date column. I assume each wrestler has only a single entry for birth_date,
# but theoretically I can see a situation in which a wrestler is listed with an incorrect birth_date that was later revised.
# So to prevent any possible errors, I am going to clean up this column.

rikishi_birth_date_list = []

for i in banzuke["birth_date"].unique():
    rikishi_birth_date_list.append(i)

rikishi_birth_date_dict = {}

for i in range(len(banzuke)):
    rikishi_birth_date_dict.update({banzuke["id"][i]: [banzuke["birth_date"][i]]})
rikishi_birth_date_dict
birth_date_df = pd.DataFrame.from_dict([([key] + value) for key, value in rikishi_birth_date_dict.items()])
birth_date_df.rename(columns={0: "id", 1: "birth_date"}, inplace=True)
# ID 7806 has no listed birth_date birth_date_df.loc[birth_date_df['id']==7806]

birth_date_df

,id,birth_date
0,1354,1955-06-01
1,4080,1953-05-16
2,4095,1953-04-03
3,4104,1952-09-29
4,4112,1957-04-26
...,...,...
5277,12833,2000-05-16
5278,12834,2005-03-02
5279,12838,2005-03-13
5280,12837,2008-02-04


In [233]:
# Now that I have a dataframes for shusshin, heya, height, weight, and birth-date I can drop them from the dataframe made from the original banzuke CSV file.
# With this new "banzuke_cleaned" dataframe, I can then merge each of the new dataframes – for shusshin, heya, height, and weight – together.
# The number of rows in banzuke_cleaned matches the number of rows in the original banzuke CSV file, which means that no rows have been accidentally
# left off during the cleanup process. 
banzuke_cleaned = banzuke.drop(["shusshin", "heya", "height", "weight", "birth_date"], axis=1)
banzuke_cleaned = pd.merge(banzuke_cleaned, shusshin_df)
banzuke_cleaned = pd.merge(banzuke_cleaned, heya_df)
banzuke_cleaned = pd.merge(banzuke_cleaned, height_df)
banzuke_cleaned = pd.merge(banzuke_cleaned, weight_df)
banzuke_cleaned = pd.merge(banzuke_cleaned, birth_date_df)
banzuke_cleaned

,id,shusshin,heya,height,avg_weight,birth_date
0,1354,Hokkaido,Kokonoe,183.0,121.3,1955-06-01
1,1354,Hokkaido,Kokonoe,183.0,121.3,1955-06-01
2,1354,Hokkaido,Kokonoe,183.0,121.3,1955-06-01
3,1354,Hokkaido,Kokonoe,183.0,121.3,1955-06-01
4,1354,Hokkaido,Kokonoe,183.0,121.3,1955-06-01
...,...,...,...,...,...,...
177873,12838,Niigata,Naruto,183.0,125.0,2005-03-13
177874,12837,Tokyo,Sadogatake,178.0,126.0,2008-02-04
177875,12837,Tokyo,Sadogatake,178.0,126.0,2008-02-04
177876,12835,Tokyo,Arashio,177.0,114.0,2001-02-10


In [234]:
# Calculate BMI
bmi_list = []

for i in range(len(banzuke_cleaned)):
    bmi = round(banzuke_cleaned.loc[i, "weight"] / ((banzuke_cleaned.loc[i, "height"] / 100) ** 2), 1)
    bmi_list.append(bmi)
bmi_list

banzuke_cleaned["bmi"] = bmi_list
# Finished calculating BMI
banzuke_cleaned

KeyError: 'weight'

In [ ]:
connection = connect(":memory:")
results.to_sql("results", connection)
banzuke.to_sql("banzuke", connection)

def sql(a_string):
    return pd.read_sql(a_string, connection)

In [ ]:
combined_sql_query = pd.read_sql_query(
    """SELECT results.basho AS 'basho', results.rikishi1_id AS 'id', results.rikishi1_shikona, banzuke.rikishi as 'rikishi', 
    results.rikishi1_rank AS 'rank', banzuke.height AS 'height', banzuke.weight AS 'weight', banzuke.bmi AS 'bmi', banzuke.heya AS 'heya',
    banzuke.shusshin AS 'shusshin', results.rikishi1_win as 'outcome', results.kimarite AS 'kimarite', results.rikishi2_id AS 'opponent id',
    results.rikishi2_shikona AS 'opponent'
    FROM results 
    JOIN banzuke 
    ON results.rikishi1_id = banzuke.id
    AND results.basho = banzuke.basho""",
    connection,
)

combined_sql_query
combined_df = pd.DataFrame(combined_sql_query)
combined_df = combined_df.drop(["rikishi1_shikona"], axis=1)

combined_df["rank"] = combined_df["rank"].astype(str)

combined_df.to_csv("data/cleaned.csv", index=False)
combined_df.head()